In [22]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [23]:
vars_df = pd.read_pickle('female_vars_testB.pkl')

with open('varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
with open('varsVARS.txt') as f:
    varsVARS = f.readlines()
    varsVARS = [l.strip('\n\r') for l in varsVARS]
    varsVARS = np.array(varsVARS)
# vars_categories = np.loadtxt("vars_categories.txt", dtype=str, delimiter='\n')
with open('vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)

In [24]:
vars_df.head()

,modality,bonf,fdr,dataframe
0,T1_nonlinear,5.442009,3.767651,idx ...
1,T1_linear,5.442009,3.650484,idx ...
2,jacobian,5.442009,3.676520,idx ...
3,vbm,5.442009,3.632807,idx ...
4,T2_nonlinear,5.442009,4.016448,idx ...


In [25]:
vars_df.iloc[0].dataframe

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
0,0,Ethnic background (0.0),Ethnic Background,0.000919,0.045977,1.000000,0.963332,0.000919,0.016224,-0.000000
1,1,Ethnic background (1.0),Ethnic Background,-0.006902,0.142788,1.000000,0.886524,0.006902,0.052309,-0.000000
2,2,Ethnic background (2.0),Ethnic Background,-0.006575,0.176541,1.000000,0.859918,0.006575,0.065543,-0.000000
3,3,Genotype measurement batch (0.0),Genetic Markers,-0.072094,3.577774,0.077575,0.000353,0.072094,3.451915,1.110277
4,4,Heterozygosity (0.0),Genetic Markers,-0.023098,1.143596,1.000000,0.252903,0.023098,0.597046,-0.000000
...,...,...,...,...,...,...,...,...,...,...
13830,17507,Manifestations of mania or irritability (0.1),Mental Health,0.004880,0.087302,1.000000,0.930486,0.004880,0.031290,-0.000000
13831,17514,Substances taken for anxiety (0.1),Mental Health,-0.006877,0.124362,1.000000,0.901105,0.006877,0.045224,-0.000000
13832,17516,Activities undertaken to treat anxiety (0.1),Mental Health,-0.131146,2.381196,0.649294,0.017834,0.131146,1.748754,0.187559
13833,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.000225,0.010113,1.000000,0.991932,0.000225,0.003518,-0.000000


In [26]:
current_mods = vars_df.modality.values
modality_names = [
    'T1 Nonlinear', 'T1 Linear', 'Jacobian', 'VBM', 'T2 Nonlinear', 'T2 Lessions', 'SWI', 'rsfMRI-0', 'rsfMRI-1',
    'rsfMRI-2', 'rsfMRI-3', 'rsfMRI-4', 'rsfMRI-5', 'rsfMRI-6', 'rsfMRI-7', 'rsfMRI-8', 'rsfMRI-9', 'rsfMRI-10', 
    'rsfMRI-11', 'rsfMRI-12', 'rsfMRI-13', 'rsfMRI-14', 'rsfMRI-15', 'rsfMRI-16', 'rsfMRI-17', 'rsfMRI-18', 'rsfMRI-19',
    'rsfMRI-20', 'rsfMRI-21', 'rsfMRI-22', 'rsfMRI-23', 'rsfMRI-24', 'tfMRI-1', 'tfMRI-2', 'tfMRI-5', 'tfMRI-COPE-1',
    'tfMRI-COPE-2', 'tfMRI-COPE-5', 'Summed Tracts', 'TBSS FA', 'TBSS ICVF', 'TBSS ISOVF', 'TBSS L1', 'TBSS L2',
    'TBSS L3', 'TBSS MD', 'TBSS MO', 'TBSS OD', 'FA', 'ICVF', 'ISOVF', 'L1', 'L2', 'L3', 'MD', 'MO', 'OD'
]
for idx, mod in enumerate(current_mods):
    vars_df.loc[vars_df["modality"] == mod, "modality"] = modality_names[idx]
    

In [27]:
ordered_cols = [
     'Modality',
     'bonf',
     'passes_bonf',
     'fdr',
     'passes_fdr',
     'idx',
     'names',
     'Categories',
     'pearson_r',
     'abs_pearson_r',
     't_test_statistic',
     'p_values',
     'p_values_corrected',
     'log_p_values',
     'log_p_values_corrected', 
]

for idx in range(len(vars_df)):
    if idx == 0:
        df = vars_df.iloc[idx].dataframe
        df['Modality'] = vars_df.iloc[idx].modality
        df['bonf'] = vars_df.iloc[idx].bonf
        df['fdr'] = vars_df.iloc[idx].fdr
        df['passes_bonf'] = df.log_p_values > df.bonf
        df['passes_fdr'] = df.log_p_values > df.fdr
        df = df[ordered_cols]
        df = df.drop(columns=['abs_pearson_r'])
    else:
        df2 = vars_df.iloc[idx].dataframe
        df2['Modality'] = vars_df.iloc[idx].modality
        df2['bonf'] = vars_df.iloc[idx].bonf
        df2['fdr'] = vars_df.iloc[idx].fdr
        df2['passes_bonf'] = df2.log_p_values > df2.bonf
        df2['passes_fdr'] = df2.log_p_values > df2.fdr
        df2 = df2[ordered_cols]
        df2 = df2.drop(columns=['abs_pearson_r'])
        df = pd.concat([df, df2])
        del df2

In [28]:
vID = []
for idx in range(len(df)):
    vID.append(varsVARS[np.where(varsHeader == df.iloc[idx].names)[0][0]])
df['Variable ID'] = vID

In [29]:
df

,Modality,bonf,passes_bonf,fdr,passes_fdr,idx,names,Categories,pearson_r,t_test_statistic,p_values,p_values_corrected,log_p_values,log_p_values_corrected,Variable ID
0,T1 Nonlinear,5.442009,False,3.767651,False,0,Ethnic background (0.0),Ethnic Background,0.000919,0.045977,0.963332,1.000000,0.016224,-0.000000,21000-0.0
1,T1 Nonlinear,5.442009,False,3.767651,False,1,Ethnic background (1.0),Ethnic Background,-0.006902,0.142788,0.886524,1.000000,0.052309,-0.000000,21000-1.0
2,T1 Nonlinear,5.442009,False,3.767651,False,2,Ethnic background (2.0),Ethnic Background,-0.006575,0.176541,0.859918,1.000000,0.065543,-0.000000,21000-2.0
3,T1 Nonlinear,5.442009,False,3.767651,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.072094,3.577774,0.000353,0.077575,3.451915,1.110277,22000-0.0
4,T1 Nonlinear,5.442009,False,3.767651,False,4,Heterozygosity (0.0),Genetic Markers,-0.023098,1.143596,0.252903,1.000000,0.597046,-0.000000,22003-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13830,OD,5.442009,False,3.399498,False,17507,Manifestations of mania or irritability (0.1),Mental Health,0.071852,1.288667,0.198445,1.000000,0.702360,-0.000000,20548-0.1
13831,OD,5.442009,False,3.399498,False,17514,Substances taken for anxiety (0.1),Mental Health,0.079972,1.450796,0.147795,0.964670,0.830339,0.015621,20549-0.1
13832,OD,5.442009,False,3.399498,False,17516,Activities undertaken to treat anxiety (0.1),Mental Health,0.005326,0.095876,0.923678,1.000000,0.034479,-0.000000,20550-0.1
13833,OD,5.442009,False,3.399498,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.022904,1.028154,0.304001,1.000000,0.517125,-0.000000,21062-0.0


In [34]:
categories = sorted(list(set(df.Categories.to_list())))

first_df = True

for cat in categories:
    df_tmp = df[(df.Categories==cat) & (df.passes_bonf==True)].sort_values('log_p_values', ascending=False)
    df_tmp = df_tmp.head(10)
    if len(df_tmp) == 0:
        continue
    if first_df == True:
        df_print = df_tmp
        first_df = False
    else:
        df_print = pd.concat([df_print, df_tmp])
        
df_print = df_print.drop(columns=[
    'bonf', 'passes_bonf', 'fdr', 'passes_fdr', 'idx', 't_test_statistic', 'p_values',
    'p_values_corrected', 'log_p_values_corrected',
])

df_print_colums_order = [
    'Modality', 'log_p_values', 'pearson_r', 'Categories', 'names', 'Variable ID'
]

df_print = df_print[df_print_colums_order]

df_print = df_print.round(decimals=3)

df_print = df_print.rename(
    columns={
        'log_p_values' : '-log(p)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description'
    }
)

df_print



,Modality,-log(p),Correlation (r),Variable Category,Variable Description,Variable ID
1773,MD,7.802,0.139,Alcohol,Frequency of consuming six or more units of al...,20416-0.0
1772,MD,7.338,0.129,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1772,rsfMRI-0,7.321,0.129,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1772,MO,7.180,0.127,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1773,L2,7.152,0.133,Alcohol,Frequency of consuming six or more units of al...,20416-0.0
1772,TBSS ISOVF,7.140,0.127,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1772,L2,7.125,0.127,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1767,L2,6.991,0.131,Alcohol,Amount of alcohol drunk on a typical drinking ...,20403-0.0
1772,FA,6.799,0.124,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1772,TBSS L3,6.563,0.121,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0


In [35]:
df_print.to_csv('Famale_top_nIDPs.csv', index=False)

In [36]:
categories = sorted(list(set(df.Categories.to_list())))

first_df = True

for cat in categories:
    df_tmp = df[(df.Categories==cat) & (df.passes_bonf==True)].sort_values('log_p_values', ascending=False)
    df_tmp = df_tmp.head(5)
    if len(df_tmp) == 0:
        continue
    if first_df == True:
        df_print = df_tmp
        first_df = False
    else:
        df_print = pd.concat([df_print, df_tmp])
        
df_print = df_print.drop(columns=[
    'bonf', 'passes_bonf', 'fdr', 'passes_fdr', 'idx', 't_test_statistic', 'p_values',
    'p_values_corrected', 'log_p_values_corrected'
])

df_print_colums_order = [
    'Modality', 'log_p_values', 'pearson_r', 'Categories', 'names', 'Variable ID'
]

df_print = df_print[df_print_colums_order]

df_print = df_print.round(decimals=3)

df_print = df_print.rename(
    columns={
        'log_p_values' : '-log(p)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description'
    }
)

df_print

df_print.to_csv('Famale_top_nIDPs_reduced.csv', index=False)

In [37]:
df_print

,Modality,-log(p),Correlation (r),Variable Category,Variable Description,Variable ID
1773,MD,7.802,0.139,Alcohol,Frequency of consuming six or more units of al...,20416-0.0
1772,MD,7.338,0.129,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1772,rsfMRI-0,7.321,0.129,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1772,MO,7.180,0.127,Alcohol,Frequency of drinking alcohol (0.0),20414-0.0
1773,L2,7.152,0.133,Alcohol,Frequency of consuming six or more units of al...,20416-0.0
3879,rsfMRI-2,6.825,-0.108,Blood Assays,C-reactive protein (0.0),30710-0.0
3879,rsfMRI-10,6.739,-0.108,Blood Assays,C-reactive protein (0.0),30710-0.0
3879,rsfMRI-5,6.716,-0.107,Blood Assays,C-reactive protein (0.0),30710-0.0
3879,rsfMRI-21,5.982,-0.101,Blood Assays,C-reactive protein (0.0),30710-0.0
3879,rsfMRI-16,5.683,-0.098,Blood Assays,C-reactive protein (0.0),30710-0.0


In [38]:
df

,Modality,bonf,passes_bonf,fdr,passes_fdr,idx,names,Categories,pearson_r,t_test_statistic,p_values,p_values_corrected,log_p_values,log_p_values_corrected,Variable ID
0,T1 Nonlinear,5.442009,False,3.767651,False,0,Ethnic background (0.0),Ethnic Background,0.000919,0.045977,0.963332,1.000000,0.016224,-0.000000,21000-0.0
1,T1 Nonlinear,5.442009,False,3.767651,False,1,Ethnic background (1.0),Ethnic Background,-0.006902,0.142788,0.886524,1.000000,0.052309,-0.000000,21000-1.0
2,T1 Nonlinear,5.442009,False,3.767651,False,2,Ethnic background (2.0),Ethnic Background,-0.006575,0.176541,0.859918,1.000000,0.065543,-0.000000,21000-2.0
3,T1 Nonlinear,5.442009,False,3.767651,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.072094,3.577774,0.000353,0.077575,3.451915,1.110277,22000-0.0
4,T1 Nonlinear,5.442009,False,3.767651,False,4,Heterozygosity (0.0),Genetic Markers,-0.023098,1.143596,0.252903,1.000000,0.597046,-0.000000,22003-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13830,OD,5.442009,False,3.399498,False,17507,Manifestations of mania or irritability (0.1),Mental Health,0.071852,1.288667,0.198445,1.000000,0.702360,-0.000000,20548-0.1
13831,OD,5.442009,False,3.399498,False,17514,Substances taken for anxiety (0.1),Mental Health,0.079972,1.450796,0.147795,0.964670,0.830339,0.015621,20549-0.1
13832,OD,5.442009,False,3.399498,False,17516,Activities undertaken to treat anxiety (0.1),Mental Health,0.005326,0.095876,0.923678,1.000000,0.034479,-0.000000,20550-0.1
13833,OD,5.442009,False,3.399498,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.022904,1.028154,0.304001,1.000000,0.517125,-0.000000,21062-0.0


In [39]:
df = df.rename(
    columns={
        'log_p_values' : '-log(P)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description',
        'bonf': 'Bonferroni Threshold',
        'passes_bonf': 'Passes Bonf.',
        'fdr': 'False Discovery Rate (FDR)',
        'passes_fdr': 'Passes FDR',
        'idx': 'Caterogy ID',
        't_test_statistic': 'T-Values',
        'p_values': 'P-Values',
        'p_values_corrected': 'FDR Adjusted P-Value',
        'log_p_values_corrected': 'FDR Adjusted -log(P)'
    }
)

df

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P),Variable ID
0,T1 Nonlinear,5.442009,False,3.767651,False,0,Ethnic background (0.0),Ethnic Background,0.000919,0.045977,0.963332,1.000000,0.016224,-0.000000,21000-0.0
1,T1 Nonlinear,5.442009,False,3.767651,False,1,Ethnic background (1.0),Ethnic Background,-0.006902,0.142788,0.886524,1.000000,0.052309,-0.000000,21000-1.0
2,T1 Nonlinear,5.442009,False,3.767651,False,2,Ethnic background (2.0),Ethnic Background,-0.006575,0.176541,0.859918,1.000000,0.065543,-0.000000,21000-2.0
3,T1 Nonlinear,5.442009,False,3.767651,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.072094,3.577774,0.000353,0.077575,3.451915,1.110277,22000-0.0
4,T1 Nonlinear,5.442009,False,3.767651,False,4,Heterozygosity (0.0),Genetic Markers,-0.023098,1.143596,0.252903,1.000000,0.597046,-0.000000,22003-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13830,OD,5.442009,False,3.399498,False,17507,Manifestations of mania or irritability (0.1),Mental Health,0.071852,1.288667,0.198445,1.000000,0.702360,-0.000000,20548-0.1
13831,OD,5.442009,False,3.399498,False,17514,Substances taken for anxiety (0.1),Mental Health,0.079972,1.450796,0.147795,0.964670,0.830339,0.015621,20549-0.1
13832,OD,5.442009,False,3.399498,False,17516,Activities undertaken to treat anxiety (0.1),Mental Health,0.005326,0.095876,0.923678,1.000000,0.034479,-0.000000,20550-0.1
13833,OD,5.442009,False,3.399498,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.022904,1.028154,0.304001,1.000000,0.517125,-0.000000,21062-0.0


In [40]:
df = df[['Modality', 'Bonferroni Threshold', 'Passes Bonf.',
       'False Discovery Rate (FDR)', 'Passes FDR', 'Caterogy ID',
       'Variable Description', 'Variable Category', 'Variable ID', 'Correlation (r)',
       'T-Values', 'P-Values', 'FDR Adjusted P-Value', '-log(P)',
       'FDR Adjusted -log(P)']]
df

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Variable ID,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
0,T1 Nonlinear,5.442009,False,3.767651,False,0,Ethnic background (0.0),Ethnic Background,21000-0.0,0.000919,0.045977,0.963332,1.000000,0.016224,-0.000000
1,T1 Nonlinear,5.442009,False,3.767651,False,1,Ethnic background (1.0),Ethnic Background,21000-1.0,-0.006902,0.142788,0.886524,1.000000,0.052309,-0.000000
2,T1 Nonlinear,5.442009,False,3.767651,False,2,Ethnic background (2.0),Ethnic Background,21000-2.0,-0.006575,0.176541,0.859918,1.000000,0.065543,-0.000000
3,T1 Nonlinear,5.442009,False,3.767651,False,3,Genotype measurement batch (0.0),Genetic Markers,22000-0.0,-0.072094,3.577774,0.000353,0.077575,3.451915,1.110277
4,T1 Nonlinear,5.442009,False,3.767651,False,4,Heterozygosity (0.0),Genetic Markers,22003-0.0,-0.023098,1.143596,0.252903,1.000000,0.597046,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13830,OD,5.442009,False,3.399498,False,17507,Manifestations of mania or irritability (0.1),Mental Health,20548-0.1,0.071852,1.288667,0.198445,1.000000,0.702360,-0.000000
13831,OD,5.442009,False,3.399498,False,17514,Substances taken for anxiety (0.1),Mental Health,20549-0.1,0.079972,1.450796,0.147795,0.964670,0.830339,0.015621
13832,OD,5.442009,False,3.399498,False,17516,Activities undertaken to treat anxiety (0.1),Mental Health,20550-0.1,0.005326,0.095876,0.923678,1.000000,0.034479,-0.000000
13833,OD,5.442009,False,3.399498,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,21062-0.0,-0.022904,1.028154,0.304001,1.000000,0.517125,-0.000000


In [41]:
df.to_csv('Famale_ALL_nIDPs.csv', index=False)

In [42]:
df_passing_FDR = df[df['Passes FDR']==True]

In [43]:
df_passing_FDR

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Variable ID,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
648,T1 Nonlinear,5.442009,False,3.767651,True,1209,"Never eat eggs, dairy, wheat, sugar (2.0)",Diet,6144-2.0,-0.079793,3.990406,0.000068,0.020866,4.168329,1.680562
2048,T1 Nonlinear,5.442009,False,3.767651,True,3316,Heel broadband ultrasound attenuation (left) (...,Skeletal Measurements,4101-2.0,-0.087045,3.822675,0.000136,0.036246,3.865720,1.440744
2051,T1 Nonlinear,5.442009,False,3.767651,True,3320,Speed of sound through heel (left) (2.0),Skeletal Measurements,4103-2.0,-0.086605,3.803203,0.000147,0.038455,3.831752,1.415049
2054,T1 Nonlinear,5.442009,False,3.767651,True,3324,"Heel quantitative ultrasound index (QUI), dire...",Skeletal Measurements,4104-2.0,-0.089740,3.941984,0.000084,0.023164,4.077190,1.635181
2076,T1 Nonlinear,5.442009,False,3.767651,True,3377,L1-L4 BMD (bone mineral density) (2.0),Skeletal Measurements,23204-2.0,-0.088822,4.129082,0.000038,0.013080,4.422315,1.883396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10768,OD,5.442009,False,3.399498,True,14352,Diagnoses - ICD10 (F009 - F00.9 Dementia in Al...,Medical History,41270-0.2713,0.078778,3.958264,0.000078,0.011928,4.110170,1.923434
10823,OD,5.442009,False,3.399498,True,14407,Diagnoses - ICD10 (G458 - G45.8 Other transien...,Medical History,41270-0.3336,0.086005,4.324022,0.000016,0.002824,4.798077,2.549193
10965,OD,5.442009,False,3.399498,True,14549,Diagnoses - ICD10 (I10 - I10 Essential (primar...,Medical History,41270-0.4052,0.074821,3.758321,0.000175,0.022412,3.757065,1.649510
11435,OD,5.442009,False,3.399498,True,15019,Diagnoses - ICD10 (M255 - M25.5 Pain in joint),Medical History,41270-0.7021,0.073118,3.672325,0.000245,0.031145,3.610170,1.506617


In [44]:
df_passing_FDR.to_csv('Famale_ALL_nIDPs_passing_FDR.csv', index=False)